In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [7]:
import os
import sys
from PIL import Image, ImageChops, ImageDraw
import random as rand
import math
from IPython import display

## 0. Utility

In [8]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def randfloat(a, b):
    return rand.random() * (b - a) + a

def my_print(s):
    sys.stdout.write(s)
    sys.stdout.flush()

## 2. Generate input

In [9]:
arrows_dir = "./masks/"

input_dir = "../input/"
create_dir(input_dir)

N = 10000


input_size = (64, 64)
arrow_size = (16, 16)

min_speed = 5
max_speed = 30
min_scale = .5
max_scale = 1.5

In [10]:
arrows = []
for arrow_name in os.listdir(arrows_dir):
    arrow_path = os.path.join(arrows_dir, arrow_name)
    image = Image.open(arrow_path)
    image = image.split()[0].convert("1")
    image = ImageChops.invert(image)
    image = image.resize(arrow_size, Image.ANTIALIAS)
    arrows.append(image)
    
arrow_cnt = len(arrows)

In [11]:
def handle_true(back_image):
    top_image = back_image.copy()
    bot_image = back_image.copy()
    top_draw = ImageDraw.Draw(top_image)
    bot_draw = ImageDraw.Draw(bot_image)
    cnts = [1, 1, 1, 2, 2, 3]
    arrow_cnt = cnts[rand.randint(0, len(cnts) - 1)]
    direction = rand.randint(0, 1) * 2 - 1
    for i in xrange(arrow_cnt):
        cur_arrow = arrows[rand.randint(0, arrow_cnt - 1)].copy()
        cur_arrow = cur_arrow.resize((int(cur_arrow.size[0] * randfloat(min_scale, max_scale)),
                                      int(cur_arrow.size[1] * randfloat(min_scale, max_scale))))
        arrow_color = (rand.randint(0, 255), rand.randint(0, 255), rand.randint(0, 255))
        angle = rand.random() * 360
        cur_arrow = cur_arrow.rotate(angle, expand=True)
        x0 = rand.randint(0, back_image.size[0] - cur_arrow.size[0] - 1)
        y0 = rand.randint(0, back_image.size[1] - cur_arrow.size[1] - 1)
        speed = rand.randint(min_speed, max_speed)
        x1 = int(x0 + direction * speed * math.cos(angle * math.pi / 180))
        y1 = int(y0 - direction * speed * math.sin(angle * math.pi / 180))
        
#         print arrow_color, x0, y0
        top_draw.bitmap((x0, y0), cur_arrow, arrow_color)
        bot_draw.bitmap((x1, y1), cur_arrow, arrow_color)
        
    full_image = Image.new("RGB", (2 * back_image.size[0], back_image.size[1]))
    full_image.paste(top_image, (0, 0))
    full_image.paste(bot_image, (back_image.size[0], 0))

    return full_image, direction
    

In [12]:
"""FOR BLACK BACKGROUNDS"""
classes = open("../labels.csv", "w+")
for i in xrange(N):
    if i % 1000 == 0:
        my_print("\r i = %i" % i)
    back_image = Image.new("RGB", input_size)
    X, y = handle_true(back_image)
    X.save(os.path.join(input_dir, str(i) + ".png"), "PNG")
    classes.write(str(i) + "," + str(y) + "\n")
    
classes.close()
    

 i = 0

AttributeError: 'tuple' object has no attribute 'copy'